In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd , sys
import matplotlib.pyplot as plt
import scikitplot as skplt
from IPython.display import clear_output
from sklearn.metrics import auc , roc_auc_score , confusion_matrix
import seaborn as sns
from sklearn.preprocessing import StandardScaler , LabelEncoder
import re , os
import tensorflow as tf
from tensorflow.contrib.layers import *
from tensorflow.contrib.tensorboard.plugins import projector
import scikitplot as skplt
from sklearn.utils.class_weight import compute_class_weight

In [2]:
sys.path.append("/home/advice/Python/SR/Custom/")
from RAdam import RAdamOptimizer


In [3]:
static = os.path.abspath("./Result")
ModelName = "effective_number"
ResultPath = os.path.join(static , ModelName)    
if tf.io.gfile.exists(ResultPath) :
    tf.io.gfile.rmtree(ResultPath)
    tf.io.gfile.makedirs(ResultPath)
else :
    tf.io.gfile.makedirs(ResultPath)

In [4]:
import optuna
from optuna.integration import SkoptSampler
import logging
import plotly
from optuna.logging import get_logger
from optuna.structs import TrialState
from optuna import type_checking
from optuna.visualization.utils import _check_plotly_availability
from optuna.visualization.utils import is_available
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf.logging.set_verbosity(tf.logging.ERROR)

/root/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning:

sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.



In [5]:
def emb(split =None, _len_=None , n = None , ratio = None , key=None) :
    print(f"Category : {key}")
    split = tf.reshape(split , shape=(-1,))
    split = tf.to_int32(split)
    if _len_ < n :
        first =  _len_
        to = _len_
        Cat = tf.one_hot(split ,depth=_len_)
        print(f"[{key}] Onehot Shape : [{first}]")
    else :
        first =  _len_
        to = int(_len_/2)
        # 2/_len_
        embeddings = tf.Variable(tf.truncated_normal([first , to], 
                                                     stddev =  0.1 ) ,
                                 dtype = tf.float32,name=key)  
        mod = sys.modules[__name__]
        setattr(mod, f'embedding_{key}',embeddings)
        Cat = tf.nn.embedding_lookup(embeddings, split)
        Cat = tf.nn.dropout(Cat , ratio)
        print(f"[{key}] Onehot Shape : [{first}] --> Embedding Shape : [{to}] ")
    return Cat


def tf_feature(X , objcol , objdict , InputdropoutRate, totalcol) :
    with tf.name_scope(f"FeatureX") :
        if objcol == [] :
            featureX = tf.nn.dropout(X , InputdropoutRate)
        else :
            featureX = EmbeddingLayer(X , objdict , InputdropoutRate, totalcol)
    return featureX

In [6]:
def tf_mish(x) :
    return x * tf.nn.tanh(tf.nn.softplus(x))

def get_weight_variable(shape, name=None,
                        type='xavier_uniform', regularize=True, **kwargs):
    initialise_from_constant = False
    if type == 'xavier_uniform':
        initial = xavier_initializer(uniform=True, dtype=tf.float32)
    elif type == 'xavier_normal':
        initial = xavier_initializer(uniform=False, dtype=tf.float32)
    elif type == 'he_normal':
        initial = variance_scaling_initializer(uniform=False, factor=2.0, mode='FAN_IN', dtype=tf.float32)
    elif type == 'he_uniform':
        initial = variance_scaling_initializer(uniform=True, factor=2.0, mode='FAN_IN', dtype=tf.float32)
    elif type == 'caffe_uniform':
        initial = variance_scaling_initializer(uniform=True, factor=1.0, mode='FAN_IN', dtype=tf.float32)
    elif type == 'simple':
        stddev = kwargs.get('stddev', 0.02)
        initial = tf.truncated_normal(shape, stddev=stddev, dtype=tf.float32)
        initialise_from_constant = True
    elif type == 'bilinear':
        weights = _bilinear_upsample_weights(shape)
        initial = tf.constant(weights, shape=shape, dtype=tf.float32)
        initialise_from_constant = True
    else:
        raise ValueError('Unknown initialisation requested: %s' % type)

    if name is None:  # This keeps to option open to use unnamed Variables
        weight = tf.Variable(initial)
    else:
        if initialise_from_constant:
            weight = tf.get_variable(name, initializer=initial)
        else:
            weight = tf.get_variable(name, shape=shape, initializer=initial)
    if regularize:
        tf.add_to_collection('weight_variables', weight)
    return weight 

def Network(number , X , dims , dropoutRate , w_init, activation) :
    with tf.variable_scope(f"Network_{number}", reuse = tf.AUTO_REUSE):
        for idx , h_dim in enumerate(dims) :
            if idx == 0 :    
                TOTAL_DIM = X.get_shape().as_list()[1]
                Weight =get_weight_variable(shape = [TOTAL_DIM , h_dim],
                                            name=f"Weight{idx}",
                                            type=w_init, regularize=True)
                Bias = tf.get_variable(f"Bias{idx}",
                                       shape = [h_dim] , dtype = tf.float32 , 
                                       initializer = tf.constant_initializer(0.0))
                Layer = activation(tf.matmul( X , Weight) + Bias)
                Layer = tf.contrib.nn.alpha_dropout(Layer , dropoutRate ) 
            else :
                Weight =get_weight_variable(shape = [dims[idx-1] ,h_dim ], 
                                            name=f"Weight{idx}",
                                            type=w_init, regularize=True)
                Bias = tf.get_variable(f"Bias{idx}",
                                       shape = [h_dim] , dtype = tf.float32 , 
                                       initializer = tf.constant_initializer(0.0))
                Layer = tf.matmul( Layer , Weight) + Bias
                if len(dims) == idx+1 :pass
                else : 
                    Layer = activation(Layer)
                    Layer = tf.contrib.nn.alpha_dropout(Layer , dropoutRate ) 
            tf.summary.histogram(f"Weight{idx}", Weight)
            tf.summary.histogram(f"Bias{idx}", Bias)
    return Layer

In [7]:
data = pd.read_csv("/home/advice/Python/SR/Data/income_evaluation.csv")
objcol = data.select_dtypes("object").columns.tolist()
data[objcol] = data[objcol].astype("category")
data.columns = [i.strip() for i in data.columns.tolist()]

In [8]:
data["income"] = data["income"].cat.codes
target = data.pop("income")
num_col = data.select_dtypes("int").columns.tolist()
cat_col = data.select_dtypes("category").columns.tolist()

In [9]:
data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
Train_X , Test_X , Train_y , Test_y =\
train_test_split(data , target , 
                 test_size = 0.3, 
                 stratify =target)

In [11]:
Train_X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
3266,30,Private,186269,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,40,United-States
6192,38,Private,258888,Some-college,10,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,35,United-States
5109,52,Private,236180,Bachelors,13,Married-spouse-absent,Other-service,Not-in-family,White,Male,0,0,50,United-States
18922,34,Local-gov,236415,Some-college,10,Never-married,Adm-clerical,Unmarried,White,Female,0,0,18,United-States
5004,36,Private,62346,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,Black,Male,0,0,40,United-States


In [12]:
for col in cat_col :
    tr_n = set(Train_X[col].unique()) 
    te_n = set(Test_X[col].unique()) 
    intersect = tr_n & te_n
    print(f"{col}, {len(tr_n)}=={len(te_n)}=={len(intersect)}?")
    if len(tr_n) != len(te_n) :
        te_need = tr_n.difference(te_n)
        tr_need = te_n.difference(tr_n)
        print("="*15)
        print(f"Warning! Check : {col}")
        print(f"Train Need Category : {tr_need}")
        print(f"Test Need Category : {te_need}")

workclass, 9==9==9?
education, 16==16==16?
marital-status, 7==7==7?
occupation, 15==15==15?
relationship, 6==6==6?
race, 5==5==5?
sex, 2==2==2?
native-country, 42==41==41?
Warning! Check : native-country
Train Need Category : set()
Test Need Category : {' Holand-Netherlands'}


In [13]:
LabelEncoding = {}
for col in cat_col :
    print(col)
    encoding = LabelEncoder()
    category = list(set(list(Train_X[col].unique()) + ["Unknown"]))
    encoding.fit(category)
    Train_X[col] = Train_X[col].cat.add_categories('Unknown')
    Train_X[col] = Train_X[col].fillna('Unknown')
    Train_X[col] = encoding.transform(Train_X[col])
    LabelEncoding[col] = encoding

workclass


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pa

education
marital-status
occupation
relationship
race
sex
native-country


In [ ]:
from time import time
start = time()
change = {}
for col in cat_col :
    change[col] = {}
    LABLEncoder = LabelEncoding[col]
    unknownlabel = set(Test_X[col].unique()).difference(set(LABLEncoder.classes_))
    c = dict(zip(unknownlabel , [np.nan] * len(unknownlabel)))
    print(c)
    c[np.nan] = 'Unknown'
    change[col] = c
    
print("{}초".format(time()- start))

{}
{}
{}
{}
{}
{}
{}
{}
0.022764205932617188초


In [ ]:
for col in cat_col :
    print(col)
    LABLEncoder = LabelEncoding[col]
    unknownlabel = set(Test_X[col].unique()).difference(set(LABLEncoder.classes_))
    c = dict(zip(unknownlabel , [np.nan] * len(unknownlabel)))
    c[np.nan] = 'Unknown'
    Test_X[col] = Test_X[col].replace(c)   
    Test_X[col] = LABLEncoder.transform(Test_X[col])

workclass


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



education
marital-status
occupation
relationship
race
sex
native-country


In [ ]:
Train_X = Train_X.astype(np.float32)
Test_X = Test_X.astype(np.float32)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_features = num_col 
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)])
clf = Pipeline(steps=[('preprocessor', preprocessor),])
clf.fit(Train_X)
Train_X[num_col] = clf.transform(Train_X)
Test_X[num_col] = clf.transform(Test_X)
#train_X[numcol] = imp_mean.transform(train_X[numcol])
#test_X[numcol] = imp_mean.transform(test_X[numcol])

In [ ]:
totalcol = data.columns.tolist()
objinfo = [idx for idx , col in enumerate(totalcol) if col in cat_col ]
objinfo

[1, 3, 5, 6, 7, 8, 9, 13]

In [ ]:
objdict = {}
for i in objinfo :
    column = totalcol[i]
    objdict[i] = len(LabelEncoding[column].classes_)
objdict

{1: 10, 3: 17, 5: 8, 6: 16, 7: 7, 8: 6, 9: 3, 13: 43}

In [ ]:
total_dim = Train_X.shape[1]

In [ ]:
tf.reset_default_graph()
batch_size = tf.placeholder(tf.int64, name="Batchsize")
X =  tf.placeholder(tf.float32 , shape = [None , total_dim])
y = tf.placeholder(tf.float32 , [None])
dropoutRate = tf.placeholder(tf.float32, name ="dropoutRate")
InputdropoutRate = tf.placeholder(tf.float32, name ="InputdropoutRate")
#y = tf.string_to_number(y)
X , y 

(<tf.Tensor 'Placeholder:0' shape=(?, 14) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>)

In [ ]:
totalcol

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']

In [ ]:
Train_X[cat_col[0]].head()

3266     4.0
6192     4.0
5109     4.0
18922    2.0
5004     4.0
Name: workclass, dtype: float32

In [ ]:
dict(zip(np.arange(len(LabelEncoding[cat_col[0]].classes_)).tolist() , 
         LabelEncoding[cat_col[0]].classes_))


{0: ' ?',
 1: ' Federal-gov',
 2: ' Local-gov',
 3: ' Never-worked',
 4: ' Private',
 5: ' Self-emp-inc',
 6: ' Self-emp-not-inc',
 7: ' State-gov',
 8: ' Without-pay',
 9: 'Unknown'}

In [ ]:
def EmbeddingLayer(X , objdict , inputratio ,totalcol ) :
    inputs = []
    for idx , key in enumerate(totalcol) :
        split = tf.slice(X , [0 , idx ] ,[-1 , 1 ] )
        if idx in objdict :
            category_n = objdict[idx]
            Cat = emb(split , category_n , 4 , inputratio , key)
            inputs.append(Cat)
        else :
            inputs.append(split)
    concatenated_layer = tf.concat(inputs, axis=1, name='concatenate') 
    return concatenated_layer


In [ ]:
TransformX = tf_feature(X , cat_col , objdict , InputdropoutRate, totalcol)

Category : workclass
[workclass] Onehot Shape : [10] --> Embedding Shape : [5] 
Category : education
[education] Onehot Shape : [17] --> Embedding Shape : [8] 
Category : marital-status
[marital-status] Onehot Shape : [8] --> Embedding Shape : [4] 
Category : occupation
[occupation] Onehot Shape : [16] --> Embedding Shape : [8] 
Category : relationship
[relationship] Onehot Shape : [7] --> Embedding Shape : [3] 
Category : race
[race] Onehot Shape : [6] --> Embedding Shape : [3] 
Category : sex
[sex] Onehot Shape : [3]
Category : native-country
[native-country] Onehot Shape : [43] --> Embedding Shape : [21] 


In [ ]:
TransformX.shape

TensorShape([Dimension(None), Dimension(61)])

In [ ]:
target_dim = len(np.unique(Train_y))
target_dim

2

In [ ]:
dims = [40 , 20, 10 , target_dim]
init = "he_normal"
activation = tf.nn.selu
Logit = Network(number= 1 ,
                X = TransformX , 
                dims = dims , 
                dropoutRate=dropoutRate,
                w_init = init,
                activation = activation)
print(Logit)
Probs = tf.nn.softmax(Logit)
target_dim = 2

Tensor("Network_1/add_3:0", dtype=float32)


In [ ]:
print(Logit)

Tensor("Network_1/add_3:0", dtype=float32)


In [ ]:
config=tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1



In [ ]:
config=tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
pd.DataFrame(sess.run(TransformX , 
                      feed_dict ={ X : Train_X.head(1).values ,
                                  InputdropoutRate : 1.0}))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1



,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,-0.632696,0.163362,0.08364,0.06562,-0.062301,0.033525,-0.037268,0.150841,0.070555,-0.040634,...,0.079562,-0.060568,-0.073257,-0.000635,0.164677,-0.13292,0.01427,-0.147333,0.066904,-0.001157


In [ ]:
pd.DataFrame(sess.run(Logit , 
                      feed_dict ={ X : Train_X.head(1).values ,
                                  InputdropoutRate : 0.8 ,
                                  dropoutRate : 0.8}))

,0,1
0,0.300647,-1.483904


## Cass-Balanced Loss Based on Effective Number of Samples


### [paper](https://arxiv.org/abs/1901.05555)


### [code](https://github.com/richardaecn/class-balanced-loss/blob/master/src/cifar_main.py)

In [ ]:
from tensorflow.losses import Reduction
no_of_classes = 2
beta = 0.9
gamma = 2.0
alpha = 1.0
y_one_hot = tf.add(alpha* tf.one_hot( tf.cast(y , tf.int32), 
                                     depth=target_dim) ,(1-alpha) / target_dim)

samples_per_cls = tf.transpose(tf.reduce_sum(y_one_hot, axis = 0 ))
pw = tf.pow(beta, samples_per_cls)
effective_num = 1.0 - tf.pow(beta, samples_per_cls)
effective_num = tf.reshape(effective_num,shape=(-1,no_of_classes))
weights = (1.0 - beta) / effective_num
weights= tf.div(weights , tf.reduce_sum(weights) * no_of_classes)
weights = weights * y_one_hot
weights = tf.reduce_sum(weights, axis = 1)
weights = tf.expand_dims(weights , axis = 1)
weights = tf.tile(weights, [1, no_of_classes])
WCE = tf.losses.softmax_cross_entropy(onehot_labels= y_one_hot , 
                                      logits= Logit , 
                                      weights= tf.reduce_mean(weights, axis=1),
                                      label_smoothing= 0.95,
                                      reduction= Reduction.MEAN
                                     )

In [ ]:
sess.run(tf.pow(beta, samples_per_cls) , 
         feed_dict ={X : Train_X.values ,
                     y : Train_y.values , 
                     InputdropoutRate : 0.9 ,
                           batch_size : 20 , 
                           dropoutRate : 0.8})

array([0., 0.], dtype=float32)

In [ ]:
# import torch
# import numpy as np
# import torch.nn.functional as F
# no_of_classes = 2
# logits = torch.rand(10,no_of_classes).float()
# labels = torch.randint(0,no_of_classes, size = (10,))
# beta = 0.9999
# gamma = 2.0
# samples_per_cls = [10,1]
# effective_num = 1.0 - np.power(beta, samples_per_cls)
# weights = (1.0 - beta) / np.array(effective_num)
# weights = weights / np.sum(weights) * no_of_classes
# labels_one_hot = F.one_hot(labels, no_of_classes).float()
# weights = torch.tensor(weights).float()
# weights = weights.unsqueeze(0)
# weights = weights.repeat(labels_one_hot.shape[0],1) * labels_one_hot
# weights = weights.sum(1)
# weights = weights.unsqueeze(1)
# weights = weights.repeat(1,no_of_classes)
# print(weights)

In [ ]:

Loss = tf.reduce_mean(WCE)
vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,scope="Network")
WEIGHTS = tf.get_collection("weight_variables")
import re 
L2 = []
for v in WEIGHTS :
    L2.append(tf.nn.l2_loss(v))
Loss += tf.add_n(L2)  * 0.0001
l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.005, scope=None)
regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, WEIGHTS )
Loss += regularization_penalty

In [ ]:
solver = RAdamOptimizer(learning_rate= 2e-5, beta1 = 0.9 ,)
solver = solver.minimize(Loss ,var_list = vars )

In [ ]:
_Loss_ = []
Epoch= 50000
mb_size = 500
print("Train")
log = logging.getLogger('TF_log')
log.setLevel(logging.DEBUG)
fileHandler = logging.FileHandler(os.path.join(ResultPath,'tf_log.txt') , mode= "w")
log.addHandler(fileHandler)

config=tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)
sess.run(tf.global_variables_initializer())
_Loss_ = []
_Epoch_ = []
_Epoch2_ = [0]
_trAUC_ , _teAUC_ = [0] , [0]


for epoch in range(Epoch) :
    idx = list(np.random.permutation(len(Train_X)))
    if (epoch > 0 ) & (epoch > 100) :
        #idx = idx + BoostingList
        idx = list(np.random.permutation(idx))
        mb_size = 1000
    else : 
        mb_size = 500
    XX = Train_X.iloc[idx, : ].values
    YY = Train_y[idx]
    batch_iter = int(len(XX) / mb_size)
    _Loss2_ = []
    for idx in range(batch_iter) :
        X_mb = XX[idx*mb_size:(idx+1)*mb_size]
        Y_mb = YY[idx*mb_size:(idx+1)*mb_size]
        Feed = {X : X_mb ,
                y : Y_mb , 
                InputdropoutRate : 0.8 ,
                dropoutRate : 0.8
               }
        _ , LOSS  = sess.run([solver , Loss] , feed_dict= Feed)
        _Loss2_.append(LOSS)
    _Loss_.append(np.mean(_Loss2_))
    _Epoch_.append(epoch)
    if (epoch > 0 ) & (epoch % 100 == 0) :
        Feed = { X : Train_X.values ,
                InputdropoutRate : 0.8 ,
                dropoutRate : 0.8
               }
        probs  = sess.run(Probs , feed_dict= Feed)
        trainDD = pd.DataFrame([Train_y ,probs[:,1]],
                          index = ["t","prob"]).T
        ### Target 1 Boosting List
        DD2 = trainDD[(trainDD.t == 1) & (trainDD.prob <0.5)]
        BoostingList = 1 * DD2.index.tolist()
        msg = "Epoch : {}  Boosting List : {}".\
              format(epoch,len(DD2.index.tolist()))
        log.info(msg)
    if (epoch > 0)  & (epoch % 100 == 0) :
        Feed = { X : Train_X.values ,
                InputdropoutRate : 0.8 ,
                dropoutRate : 0.8
               }
        clear_output()
        tr_real_target = np.squeeze(Train_y.values)
        tr_probs  = sess.run(Probs , feed_dict= Feed)
        tr_AUC = roc_auc_score(tr_real_target ,  tr_probs[:,1])
        trainDD = pd.DataFrame([Train_y.values ,probs[:,1]],
                          index = ["t","prob"]).T
        ####################################################
        Feed = {X : Test_X.values  ,
                InputdropoutRate : 0.8 ,
                dropoutRate : 0.8
           }
        te_probs  = sess.run(Probs , feed_dict= Feed)
        te_real_target = np.squeeze(Test_y.values)
        te_AUC = roc_auc_score(te_real_target , te_probs[:,1])
        testDD = pd.DataFrame([te_real_target ,te_probs[:,1]],
                          index = ["t","prob"]).T
        
        fig , axes = plt.subplots(nrows=4 ,ncols=2,
                  figsize=(15,12) )
        plt.subplots_adjust(left=0.05, bottom=0.1, right=0.99, 
                            top=0.95, wspace=None, hspace=0.5)
        ax = axes.flatten()
        sns.boxplot(x="t", y="prob", data=trainDD, ax = ax[0])
        ax[0].set_title("train : {:.3f}".format(tr_AUC), fontsize= 25)
        sns.boxplot(x="t", y="prob", data=testDD, ax = ax[1])
        ax[1].set_title("test : {:.3f}".format(te_AUC), fontsize= 25)
        skplt.metrics.plot_ks_statistic(Train_y.values, tr_probs , 
                                        ax = ax[2] , 
                                        title = "[Train] KS Static PLOT")
        skplt.metrics.plot_ks_statistic(Test_y.values, te_probs ,
                                        ax = ax[3], 
                                        title = "[Test] KS Static PLOT")

        ax3 = plt.subplot(413)
        plt.subplots_adjust(left=0.05, bottom=0.1, right=0.99, 
                            top=0.95, wspace=None, hspace=0.7)
        ax3.plot(_Epoch_ , _Loss_ )
        ax3.set_title(msg, fontsize= 20)
        _Epoch2_.append(epoch)
        _trAUC_.append(tr_AUC)
        _teAUC_.append(te_AUC)
        ax4 = plt.subplot(414)
        ax4.plot(_Epoch2_ , _trAUC_ , label = "train auc")
        ax4.plot(_Epoch2_ , _teAUC_ , label = "test auc")
        ax4.legend()
        msg = "Epoch : {} / AUC | Train : {:.2f} Test : {:.2f}".format(epoch , 
                                                               100*tr_AUC ,
                                                               100*te_AUC
                                                              )
        ax4.set_title(msg, fontsize= 20)
        savefig = os.path.join(ResultPath,f'plot.{epoch:04d}.png')
        plt.savefig(savefig)
        plt.show()
        

Train
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1



/root/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:1143: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike



In [ ]:
print("end")

In [ ]:

def create_model(trial = None) :
    nodes_1 = trial.suggest_int("nodes_1",30,65)
    nodes_2 = trial.suggest_int("nodes_2",30,65)
    nodes_3 = trial.suggest_int("nodes_3",10,30)
    
    init_candidate =\
    ["xavier_uniform","xavier_normal", 
     "he_normal", "he_uniform","caffe_uniform"]
    activate_candidate = \
    [tf.nn.selu, tf_mish , tf.nn.leaky_relu , tf.nn.elu , tf.nn.relu ]
    activate_int = [0,1,2,3,4]
    
    target_n = 2 
    init =\
    trial.suggest_categorical('weight_init', init_candidate)
    selected =\
    trial.suggest_categorical('activation', activate_int)
    activation = activate_candidate[selected]
    alpha = trial.suggest_uniform('smoothLabel', 0.7, 0.99)
    ##################################################################
    tf.reset_default_graph()
    batch_size = tf.placeholder(tf.int64, name="Batchsize")
    X =  tf.placeholder(tf.float32 , shape = [None , total_dim])
    y = tf.placeholder(tf.float32 , [None])
    regularizer = tf.placeholder(tf.float32, name ="regularizer")
    dropoutRate = tf.placeholder(tf.float32, name ="dropoutRate")
    InputdropoutRate = tf.placeholder(tf.float32, name ="InputdropoutRate")
    ##################################################################
    TransformX = tf_feature(X , cat_col , objdict , InputdropoutRate, totalcol)
    dims = [nodes_1 , nodes_2, nodes_3 , target_dim]
    print(dims)
    Logit = Network(number= trial.number ,
                    X = TransformX , 
                    dims = dims , 
                    dropoutRate=dropoutRate,
                    w_init = init,
                    activation = activation)
    Probs = tf.nn.softmax(Logit)
    yy = tf.one_hot( tf.cast(y , tf.int32), depth=target_dim) 
    y_one_hot = tf.add(alpha* yy,(1-alpha) / target_dim)
    from sklearn.utils.class_weight import compute_class_weight
    y_weight_info = compute_class_weight(
        class_weight= "balanced" , 
        classes = np.unique(Train_y),
        y= np.squeeze(Train_y))

    weight = tf.constant([ y_weight_info[1] ] ) #
    WCE = tf.nn.weighted_cross_entropy_with_logits(targets = y_one_hot ,
                                                   logits = Logit , 
                                                   pos_weight =  weight)

    
    Loss = tf.reduce_mean(WCE)
    vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                             scope=f"Network_{trial.number}")
    WEIGHTS = tf.get_collection("weight_variables")
    L2 = []
    for v in WEIGHTS :
        L2.append(tf.nn.l2_loss(v))
    Loss += tf.add_n(L2)  * regularizer
    l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.005, scope=None)
    regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, WEIGHTS )
    Loss += regularizer * regularization_penalty
    kwargs = {}
    kwargs['learning_rate'] = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    optimizer = RAdamOptimizer(**kwargs)
    solver = optimizer.minimize(Loss ,var_list = vars )

    prediction = tf.argmax(Probs, 1)
    correct = tf.argmax(y_one_hot, 1)
    equality = tf.equal(prediction, correct)
    accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))
    tf.summary.scalar(f"Accuracy_{trial.number}",accuracy)
    tf.summary.scalar(f"Loss_{trial.number}",Loss)
    tf.summary.histogram(f"Probability_{trial.number}", Probs)
    merged = tf.summary.merge_all()
    return [Probs , Loss , solver, 
            X , y , dropoutRate , InputdropoutRate ,regularizer,
            merged]
    

In [ ]:
best_auc = 0.0
log = logging.getLogger('optuna')
log.setLevel(logging.DEBUG)
fileHandler = logging.FileHandler(
    os.path.join(ResultPath,'optuna_log.txt') , mode= "w")
log.addHandler(fileHandler)


def objective(trial):   
    Lists = create_model(trial =trial)
    (Probs , Loss , solver, 
     X , y , dropoutRate , InputdropoutRate , regularizer,
     merged
    ) = Lists
    
    if trial.number > 11 :
        Epoch = 1000
        mb_size = 1000
    else :
        Epoch = 500
        mb_size = 500
    config=tf.ConfigProto(log_device_placement=True)
    config.gpu_options.allow_growth = True
    sess = tf.Session(config = config)
    train_writer = tf.summary.FileWriter(os.path.join(ResultPath,
                                                      f'train_{trial.number}'),
                                         sess.graph)
    saver = tf.train.Saver()
    config = projector.ProjectorConfig()
    mod = sys.modules[__name__]
    for cat in cat_col :
        if len(LabelEncoding[cat].classes_) < 5 :
            continue
        index2word_map = dict(zip(np.arange(len(LabelEncoding[cat].classes_)).tolist() , 
                                 LabelEncoding[cat].classes_))
        metadata_file = os.path.join(ResultPath, f'train_{trial.number}', 
                               f'metadata_{trial.number}_{cat}.tsv') 
        with open(metadata_file, "w") as metadata:
            metadata.write('Name\tClass\n')
            for k, v in index2word_map.items():
                metadata.write('%s\t%d\n' % (v, k))
        embedding = config.embeddings.add()
        embedding.tensor_name = getattr(mod,  f'embedding_{cat}').name
        # Link this tensor to its metadata file (e.g. labels).
        embedding.metadata_path = metadata_file
    projector.visualize_embeddings(train_writer, config)

    
    sess.run(tf.global_variables_initializer())
    _Loss_ = []
    _Epoch_ = []
    _Epoch2_ = [0]
    _trAUC_ , _teAUC_ = [0] , [0]
    for epoch in range(Epoch) :
        if epoch < 100 :
            regularizer_rate = 1e-5
        elif epoch < 200 :
            regularizer_rate = 1e-4
        elif epoch < 400 :
            regularizer_rate = 1e-3
        elif epoch < 800 :
            regularizer_rate = 1e-2
        idx = list(np.random.permutation(len(Train_X)))
        XX = Train_X.iloc[idx, : ].values
        YY = Train_y[idx]
        batch_iter = int(len(XX) / mb_size)
        _Loss2_ = []
        for idx in range(batch_iter) :
            X_mb = XX[idx*mb_size:(idx+1)*mb_size]
            Y_mb = YY[idx*mb_size:(idx+1)*mb_size]
            Feed = {X : X_mb ,
                    y : Y_mb , 
                    regularizer : regularizer_rate,
                    InputdropoutRate : 0.8 ,
                    dropoutRate : 0.8
                   }
            _ , LOSS  = sess.run([solver , Loss] , feed_dict= Feed)
            _Loss2_.append(LOSS)
        _Loss_.append(np.mean(_Loss2_))
        _Epoch_.append(epoch)
        if epoch % 100 == 0 :
            saver.save(sess, os.path.join(ResultPath, f'train_{trial.number}', "model.ckpt" ), epoch)
            Feed = {X : Test_X.values  ,
                    y : Test_y.values ,
                    regularizer : regularizer_rate,
                    InputdropoutRate : 0.8 ,
                    dropoutRate : 0.8,
                   }
            summary = sess.run( merged ,feed_dict= Feed)
            train_writer.add_summary(summary, epoch)
            
    train_writer.close()
    Feed = {X : Test_X.values  ,
                InputdropoutRate : 0.8 ,
                dropoutRate : 0.8
           }
    te_probs  = sess.run(Probs , feed_dict= Feed)
    te_real_target = np.squeeze(Test_y.values)
    AUC = roc_auc_score(te_real_target , te_probs[:,1])
    global best_auc
    if best_auc < AUC :
        log.info("Try : {}, {} < {}".format(trial.number , best_auc ,AUC))
        best_auc= AUC
    clear_output()
    return AUC
        

In [ ]:
optuna.logging.enable_propagation()
optuna.logging.disable_default_handler()
sampler = SkoptSampler(
    skopt_kwargs={'n_random_starts':0,
                  'acq_func':'EI',
                  'acq_func_kwargs': {'xi':0.02, 
                                      "x0" : None, 
                                      "y0" : None}})
if __name__ == '__main__':
    logging.getLogger().info("Start optimization.")
    study = optuna.create_study(direction='maximize',
                               sampler = sampler)
    study.optimize(objective, 
                   n_trials=100,
                   n_jobs = 10
                  )
    print('Number of finished trials: ', len(study.trials))

    print('Best trial:')
    trial = study.best_trial

    print('  Value: ', trial.value)

    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))


Category : workclass
Category : workclass
Category : workclass
Category : workclassCategory : workclass
Category : workclassCategory : workclassCategory : workclass



Category : workclassCategory : workclass

[workclass] Onehot Shape : [10] --> Embedding Shape : [5] 
[workclass] Onehot Shape : [10] --> Embedding Shape : [5] 
Category : education
Category : education
[workclass] Onehot Shape : [10] --> Embedding Shape : [5] [workclass] Onehot Shape : [10] --> Embedding Shape : [5] 

[workclass] Onehot Shape : [10] --> Embedding Shape : [5] 
[workclass] Onehot Shape : [10] --> Embedding Shape : [5] 
[workclass] Onehot Shape : [10] --> Embedding Shape : [5] Category : educationCategory : education


[workclass] Onehot Shape : [10] --> Embedding Shape : [5] [workclass] Onehot Shape : [10] --> Embedding Shape : [5] Category : education


[workclass] Onehot Shape : [10] --> Embedding Shape : [5] Category : educationCategory : education


[education] Onehot Shape : [17] --> Embedding Shape :

/root/anaconda3/lib/python3.6/site-packages/pandas/core/series.py:1143: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike

